In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
from scipy.special import gamma, gammaincc, hyp1f1, erf
from scipy import integrate, optimize
from scipy.integrate import odeint, solve_ivp, trapz, quad, quad_vec, simps
from mpmath import ei, hyper
import random
from matplotlib.pyplot import cm
from tqdm import tqdm
import matplotlib as mpl
import time
import warnings
import gc
import math
from datetime import datetime
mp, me = 938.28*1e6, 510998 #in eV
ms = mp

In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')
mpl.rcParams['figure.dpi']= 150
plt.rcParams["font.family"] = "Times New Roman"
mpl.rcParams['text.usetex'] = True

## Funtions for $\tilde \gamma_n$ in terms of symmetric and anti-symmetric parts:

In [ ]:
def get_x_sym(func, ui, uf, m1_m2, *args):
    x_sym = (func(ui, uf, m1_m2, *args) + func(uf, ui, m1_m2, *args)) / 2
    return x_sym

def get_x_asym(func, ui, uf, m1_m2, *args):
    x_asym = (func(ui, uf, m1_m2, *args) - func(uf, ui, m1_m2, *args)) / 2
    return x_asym

def get_g0if(ui, uf, m1_m2):
    m2_m1 = 1/m1_m2
    
    g0if = np.exp(-uf**2/2) * erf(np.sqrt(m1_m2 / 8) * ((ui - uf) + m2_m1*(ui + uf)) )
    return g0if

def get_g2if(ui, uf, m1_m2, args):
    mchi_ms = args
    m2_m1 = 1/m1_m2
    ms_mchi = 1/mchi_ms
    ms_M = 1/(mchi_ms+1) #ms/(mchi+ms)
    mchi_M = 1/(1+ms_mchi)
    
    g0if = get_g0if(ui, uf, m1_m2)
    g2if = (1 + 2/mchi_M + ms_mchi * uf**2) * g0if
    return g2if

def get_h2if_pm(ui, uf, m1_m2, args):
    pm1, mchi_ms = args
    """pm1: +1 or -1"""
    m2_m1 = 1/m1_m2
    ms_mchi = 1/mchi_ms
    ms_M = 1/(mchi_ms+1) #ms/(mchi+ms)
    mchi_M = 1/(1+ms_mchi)

    h2if_pm = np.exp(-uf**2/2) * (1/ms_M*(ui + pm1*uf) + 2*pm1*uf) \
                * np.exp( -m1_m2/8 * ((ui-uf) + m2_m1*(ui+uf))**2 )
    return h2if_pm

def get_p4if(ui, uf, m1_m2, args):
    mchi_ms = args
    m2_m1 = 1/m1_m2
    ms_mchi = 1/mchi_ms
    ms_M = 1/(mchi_ms+1) #ms/(mchi+ms)
    mchi_M = 1/(1+ms_mchi)
    
    g0if = get_g0if(ui, uf, m1_m2)
    
    p4if = g0if * ( 15*mchi_ms + 10*(uf**2 + 2) + ms_mchi*(uf**4 + 4*uf**2 + 8) )
    return p4if

def get_h4if_pm(ui, uf, m1_m2, args):
    pm1, mchi_ms = args
    """pm1: +1 or -1"""
    m2_m1 = 1/m1_m2
    ms_mchi = 1/mchi_ms
    ms_M = 1/(mchi_ms+1) #ms/(mchi+ms)
    mchi_M = 1/(1+ms_mchi)
    
    h2if_pm = get_h2if_pm(ui, uf, m1_m2, [pm1, mchi_ms])
    h4if_pm = ( 1 + 2/mchi_M + ms_mchi*uf**2 ) * h2if_pm
    return h4if_pm

def get_k4if_pm(ui, uf, m1_m2, args):
    pm1, mchi_ms = args
    """pm1: +1 or -1"""
    m2_m1 = 1/m1_m2
    ms_mchi = 1/mchi_ms
    ms_M = 1/(mchi_ms+1) #ms/(mchi+ms)
    mchi_M = 1/(1+ms_mchi)
    
    h2if_pm = get_h2if_pm(ui, uf, m1_m2, [pm1, mchi_ms])
    k4if_pm = 4*np.exp(-uf**2/2) * ( 1/ms_M*(ui + pm1*uf) + 6*pm1*uf )\
                            * np.exp( -m1_m2/8 * ((ui-uf) + m2_m1*(ui+uf))**2 )\
              + h2if_pm * (1/ms_M*(ui + pm1*uf) + 2*pm1*uf) * (ms_mchi*(ui - pm1*uf) + (ui + pm1*uf))
    return k4if_pm
    
def get_gamma0(ms_mchi, ui, uf):
    mchi_ms = 1/ms_mchi
    ms_M = 1/(mchi_ms+1) #ms/(mchi+ms)
    mchi_M = 1/(1+ms_mchi)
    
    g0if_sym_msmchi = get_x_sym(get_g0if, ui, uf, ms_mchi)
    g0if_asym_mchims = get_x_asym(get_g0if, ui, uf, mchi_ms)
    gamma0 = 2*np.sqrt(1/ms_M) * (g0if_sym_msmchi + g0if_asym_mchims)
    return gamma0

def get_gamma2(ms_mchi, ui, uf):
    mchi_ms = 1/ms_mchi
    ms_M = 1/(mchi_ms+1) #ms/(mchi+ms)
    mchi_M = 1/(1+ms_mchi)
    
    g2if_sym_msmchi = get_x_sym(get_g2if, ui, uf, ms_mchi, mchi_ms)
    g2if_asym_mchims = get_x_asym(get_g2if, ui, uf, mchi_ms, mchi_ms)
    h2if_pm_sym_msmchi = get_x_sym(get_h2if_pm, ui, uf, ms_mchi, [+1, mchi_ms])
    h2if_pm_asym_mchims = get_x_asym(get_h2if_pm, ui, uf, mchi_ms, [-1, mchi_ms])
    
    gamma2 = 2 * np.sqrt(mchi_ms/ms_M) * ( np.sqrt(mchi_ms)*(g2if_sym_msmchi + g2if_asym_mchims) \
                                          - 1/np.sqrt(2*np.pi) * (h2if_pm_sym_msmchi + h2if_pm_asym_mchims)
                                         )
    return gamma2

def get_gamma4(ms_mchi, ui, uf):
    mchi_ms = 1/ms_mchi
    ms_M = 1/(mchi_ms+1) #ms/(mchi+ms)
    mchi_M = 1/(1+ms_mchi)
    
    p4if_sym_msmchi = get_x_sym(get_p4if, ui, uf, ms_mchi, mchi_ms)
    p4if_asym_mchims = get_x_asym(get_p4if, ui, uf, mchi_ms, mchi_ms)
    h4if_pm_sym_msmchi = get_x_sym(get_h4if_pm, ui, uf, ms_mchi, [+1, mchi_ms])
    h4if_pm_asym_mchims = get_x_asym(get_h4if_pm, ui, uf, mchi_ms, [-1, mchi_ms])
    k4if_pm_sym_msmchi = get_x_sym(get_k4if_pm, ui, uf, ms_mchi, [+1, mchi_ms])
    k4if_pm_asym_mchims = get_x_asym(get_k4if_pm, ui, uf, mchi_ms, [-1, mchi_ms])
    
    gamma4 = mchi_ms**2 / (4*np.sqrt(np.pi*mchi_M)) \
                            * ( 8*np.sqrt(np.pi*ms_mchi) * (p4if_sym_msmchi + p4if_asym_mchims)
                               - 8*np.sqrt(2) * (h4if_pm_sym_msmchi + h4if_pm_asym_mchims) 
                               - np.sqrt(2) * (k4if_pm_sym_msmchi + k4if_pm_asym_mchims)
                              )
    return gamma4

def get_gamma_n(n, ms_mchi, ui, uf):
    mchi_ms = 1/ms_mchi
    if n==0:
        return get_gamma0(ms_mchi, ui, uf)

    elif n==2:
        return get_gamma2(ms_mchi, ui, uf)

    elif n==4:
        return get_gamma4(ms_mchi, ui, uf)

In [ ]:
"""define constants:"""
c = 3e10 #cm/s
planck_h = 4.136e-15 #in eV*s
planck_hbar = 6.582e-16 #in eV*s
kB = 8.617e-5 #Boltzmann constant = eV_per_K
alpha_EM = 1/137 #fine structure constant
mp, me = 938.28*1e6, 510998 #in eV
kmperMpc = 3.24e-20

# cosmological parameters:
h, Om_r, Om_m, Om_de = 0.673, 9.2e-5, 0.315, 0.686
fchi = 1 # Omega_chi/Omega_DM
fHe = 0.08 #number ratio of He:H nuclei
Om_DM = 0.22 #Omega_dm for ALL dm, taken from HyRec2 input.dat default
Om_b = 0.04 #Omega for baryons, taken from HyRec2 input.dat default
Om_chi = fchi * Om_DM
YHe = 0.245 #taken from HyRec2 input.dat default, used to calculate nH
rho_crit = 1.053e4* h**2 # eV/cm^3, taken from https://pdg.lbl.gov/2019/reviews/rpp2019-rev-astrophysical-constants.pdf
rho_b0 = rho_crit * Om_b
rho_chi0 = rho_crit * Om_chi
nb0 =  1.13e-5 * Om_b *h**2 # cm^-3 ,nb today
nH0 = 11.22e-6*Om_b*h**2*(1. - YHe) # cm^-3 ,nH today taken from HyRec2 history.c rec_get_cosmoparam

In [ ]:
def get_Tr(a): #returns CMB temperature in eV
    eVperK = 8.622e-5
    Tr = (2.73 * eVperK) / a
    return Tr

def get_H(a): #returns Hubble rate in 1/s
    Hsq = (h * 100*kmperMpc)**2 * ( Om_r / a**4 + Om_m / a**3 + Om_de ) #in s^(-1)
    return np.sqrt(Hsq) # 1/s

def get_d_Delta_f_dlny(lny, Delta_f
                       , n, ms_mchi, u_array, dlnu, Mij, collisions_on): #dim: [ui_arr]
    y = np.exp(lny)

    if ms == mp and collisions_on == 1 : # if ms = mp, and DM-b collisions are ON/allowed
        mchi_ms = 1/ms_mchi
#~~~~~~~~~~~
#         C_over_H_coeff = 1/(2*get_cn(n))/y**((n+3)/2) \
#                     *(1+ms_mchi)/(1+mchi_ms)**((n-1)/2)
#         C_delta_f_over_H = C_over_H_coeff * np.einsum('ij,j->i', Mij, Delta_f)
#~~~~~~~~~~~
        C_delta_f_over_H = np.einsum('ij,j->i', Mij, Delta_f)/y**((n+3)/2)
    elif collisions_on == 0: # collisions are off, only redshifting 
        C_delta_f_over_H = 0
    
    feq = 4*np.pi / (2*np.pi)**(3/2) * u_array**2 * np.exp(-u_array**2/2)
    f = feq + Delta_f
    F = u_array* f # define F = u*f
    """"""
    du_array= u_array*dlnu
    G = get_gradient(F, dlnu)  # partial f/partial lnu
    """explicitly set edges so as to ensure number conservation:"""
    G[0] = (F[0]+F[1])/(2*dlnu)   # explicitly setting edges
    G[-1] = -(F[-1]+F[-2])/(2*dlnu) # explicitly setting edges
    """    """
    G = G / u_array#convert from dF/dlnu to dF/du
    d_Delta_f_dlny = C_delta_f_over_H + G/2

    return d_Delta_f_dlny
    
def get_gradient(f, dx):
    grad_f = np.zeros(len(f))
    for i in range(1, len(f)-1):
        grad_f[i] = (f[i+1]-f[i-1]) / (2*dx)
    return grad_f

def get_initial_Delta_f(n, ms_mchi
                        , u_array, dlnu
                        , y_array, dlny
                        , X_i = -1):
    # X is defined as Tchi/Tb
    if X_i < 0: # if a valid X_i = Tchi_i/Tb_i was not externally provided, it needs to be calculated for ai << adec:
        eps = (y_array[0])**((n+3)/2)
        X_i = (1 - eps + (n + 5 + 4*ms_mchi)/2/(1+ms_mchi) * eps**2) #--> Paper-I eq 93
    """if X_i is externally provided, leave it unchanged"""
    """return Delta_f0 as MB at that X_i"""
    feq = 4*np.pi * u_array**2/(2*np.pi)**(3/2) \
                                        * np.exp(-u_array**2/2)
    fMB_Tchi_i = 4*np.pi /X_i**(3/2) * u_array**2 \
                                / (2*np.pi)**(3/2) * np.exp(- u_array**2/2/X_i)
    return fMB_Tchi_i - feq

In [ ]:
def get_Mij_and_Gamma_ij(n, ms_mchi, u_array, dlnu):
    """
    mchi: mass of DM particle in eV
    ms: mass of scatterer in eV
    n: index of the power-law scattering cross-section, sigma(v) = sigma_n * v^n
    """
    start_time = time.time()

    mchi_ms = 1/ms_mchi
    ms_M = 1/(mchi_ms+1) #ms/(mchi+ms)
    mchi_M = 1/(1+ms_mchi)  #mchi/(mchi+ms)
    du_array = u_array*dlnu
    nu = len(u_array)
    
    gamma_n0 = get_gamma_n(n, ms_mchi, u_array[:,None], u_array[None,:])
    """symmetrize gamma_n retaining only the elements in the upper triangle where ui<uf:"""
    gamma_n = np.triu(gamma_n0) + np.tril(gamma_n0.T,k=-1)
    Gamma = u_array[None,:]/u_array[:,None] \
                    * np.exp(+u_array[:,None]**2 / 2) * gamma_n #1 / mchi_M / np.sqrt(ms_M) / 4 \
#                 * 
    """Gamma --> Mij"""
    Mij_coeff = 1/8/get_cn(n) * ms_M**(n/2 - 1)/mchi_M**2
    Mij = Mij_coeff*( np.einsum('ij, j->ij' , Gamma.T, du_array)\
                        - np.einsum('ij, i -> ij',
                                    np.identity(nu),
                                    np.einsum('ik, k -> i', Gamma, du_array) 
                                    )
                    )
    """"""    
    """TESTS:"""
    feq = 4*np.pi / (2*np.pi)**(3/2) * u_array**2 * np.exp(- u_array**2 / 2)
    test0 = np.abs(np.einsum('ij,j-> i', Mij, feq))
    norm0 = np.einsum('ij,j-> i', np.abs(Mij), feq)
    top10 = np.sort(test0/norm0)[-10:]
    if np.any(test0/norm0 > 1e-5):
        print('********************Mij FAILED TEST:\n sum_j(Mij feq_j)/norm > 1e-5\n********************')
        print("\ntop 10 values of sum_j(Mij feq_j)/norm::: "+str(top10))
        return None
    elif np.any(top10>1e-10):
        print("\ntop 10 values of sum_j(Mij feq_j)/norm::: "+str(top10))
    
    test1 = np.abs(np.einsum('ij, i -> j', Mij, du_array))
    norm1 = np.einsum('ij, i -> j', np.abs(Mij), du_array)
    top10 = np.sort(test1/norm1)[-10:]
    if np.any(test1/norm1 > 1e-5):
        print('********************Mij FAILED TEST:\n sum_i(Mij)/norm > 1e-5\n********************')
        print("\ntop 10 values of sum_i(Mij)/norm::: "+str(top10))
        return None
    elif np.any(top10>1e-10):
        print("\ntop 10 values of sum_i(Mij)/norm::: "+str(top10))
    """"""
    return Mij, Gamma

def get_Delta_f(n, ms_mchi
                        , u_array, dlnu 
                        , Mij
                        , y_array, dlny
                        , rtol = 1e-3, atol = 1e-5
                        , collisions_on = 1 # 1: DM-b collisions are allowed by default; 0: only redshifting, no DM-b collisions
                        , X_i = 0.95): 
    
    feq = (4*np.pi) / (2*np.pi)**(3/2) * u_array**2 * np.exp(- u_array**2 / 2)
    ny = len(y_array)
    start_time = time.time()
    if collisions_on == 1: #if DM-b collisions are allowed, 
#                             calc initial Delta_f at ai << adec
        Delta_f0 = get_initial_Delta_f(n, ms_mchi
                                       , u_array, dlnu
                                       , y_array, dlny) #initial delta_f
    elif collisions_on == 0: # i.e., only redshifting, no DM-b collisions
        # then: 
        # (i) Mij is already a zero matrix [nu x nu] (see solve_Delta_f_given_n() call for the no collisions case in accuracy_test_forced_MB.ipynb)
        # (ii) set initial Delta_f to be MB at some Tchi_i < Tb_i (Tchi_i = X_i * Tb_i, X_i = 0.95 by default)
        # (iii) get Delta_f0 as MB at that X_i
        Delta_f0 = get_initial_Delta_f(n, ms_mchi
                                       , u_array, dlnu
                                       , y_array, dlny
                                       , X_i) # (iii) above

    """solve the ODE:"""
    print("starting ODE solver:")
    ysol = solve_ivp(get_d_Delta_f_dlny # function for ODE
                     , (np.log(y_array[0]), np.log(y_array[-1])) #time range
                     , Delta_f0 # initial condition
                     , args = (n, ms_mchi, u_array, dlnu, Mij, collisions_on) 
                     , t_eval = np.log(y_array) #t_eval ensures output at y_arr values
                     , method = 'BDF'
                     , rtol = rtol, atol = atol
                     , max_step = dlny # need to provide max_step, or the solver gets lost
                    )

    Delta_f = ysol.y #dimension = [u_array, a_arr]

    end_time = time.time()
    print('$\Delta f$ calculated.\nRun time: %i min, OR %0.3E sec'% 
            (int((end_time-start_time)/60),  (end_time-start_time) )
         )
    """test:"""
    du_array = u_array*dlnu
    test = abs(np.einsum('ij,i->j',Delta_f,du_array))
    norm = np.einsum('ij,i->j',abs(Delta_f),du_array)
    top10_inds = np.argsort(test/norm)[-10:]
    top10 = (test/norm)[top10_inds]
    if np.any(test/norm > 1e-5):
        print("Delta_f failed test: test/norm > 1e-5")
        
    if Delta_f.shape[1]!=len(y_array):
        print("Delta_f failed test: Delta_f.shape[1] != len(y_arr)")

    else: print("Delta_f passed tests")
    return Delta_f, top10, top10_inds

In [ ]:
"""functions for temperatures & heating rates:"""
def get_cn(n):
    """ eq (40) in Paper-I"""
    cn = 2**( (n+5)/2 ) / ( 3*np.sqrt(np.pi) ) * gamma(n/2 + 3)
    return cn
def get_beta_n(x, n):
    """ eq (39) in Paper-I,
    used in get_dimensionless_heating_rate_exact below"""
    beta_n = hyp1f1( -(n+3)/2, 3/2, -(x**2)/2 )
    return beta_n

def get_alpha_n(x, n):
    """ eqs(38) in Paper-I,
    used in get_dimensionless_heating_rate_exact below"""
    alpha_n = hyp1f1( -(n+1)/2, 5/2, -(x**2)/2 )
    return alpha_n

def get_dyXMB_dy(ms_mchi, n, y, yX): 
    """eq(92) in Paper-I,
    used in get_dimensionless_Q_and_X_MB below"""
    return ((1 + ms_mchi *yX/y)/(1 + ms_mchi))**((n+1)/2) *(1 - yX/y)/y**((n+3)/2) 

def get_partial_dyX_dy(ms_mchi, n, y, yX): # \partial[d(yX)/dy] / \partial(yX)
    """used in get_dimensionless_Q_and_X_MB below"""
    stuff = (1 + ms_mchi *yX/y)/(1 + ms_mchi)
    result = - stuff**((n+1)/2)/y**((n+5)/2)
    result += (n+1)/2 * ms_mchi/(1+ms_mchi) *stuff**((n-1)/2) *(1 - yX/y)/y**((n+5)/2)
    return result

def get_dimensionless_Q_and_X_MB(n, ms_mchi, y_array, dlny): 
    """returns dimensionless heating rate Q^{MB} = \dot{Q}_\chi/(3/2 H T_b) 
    and
    dimensionless temperature ratio X^{MB} = T_\chi^{MB}/T_b
    in the MB approximation"""
    y = y_array[0]
    #initiate XMB with asymptotic value, eq(93) in Paper-I:
    XMB_arr = np.array( [1 - y**((n+3)/2) + (n+5 + 4*ms_mchi)/2/(1+ms_mchi) * y**(n+3)] ) 
    #initial QMB = d(y*XMB)/dy:
    QMB_arr = np.array( [1 - (n+5)/2 * y**(n+3)/2 
                     + (n+4)*(n+5 + 4*ms_mchi)/2/(1 + ms_mchi) * y**(n+3)] )
    yXMB = y*XMB_arr[0] #calculated with quantities from previous step [i-1]

    for i in range(1,len(y_array)):
        dy = y*(np.exp(dlny) - 1.) 
        y *= np.exp(dlny)
        
        partial_dyXMB_dy = get_partial_dyX_dy(ms_mchi, n, y, yXMB)
        dyXMB_dy = get_dyXMB_dy(ms_mchi, n, y, yXMB)
        QMB = dyXMB_dy / (1 - partial_dyXMB_dy * dy)
        
        yXMB   += QMB*dy #update to step [i] quantities to calculate XMB_arr[i] in next line
        XMB    = yXMB/y
        QMB_arr = np.append(QMB_arr, QMB)
        XMB_arr = np.append(XMB_arr, XMB)
    
    return QMB_arr, XMB_arr

def get_dimensionless_heating_rate_exact(n, ms_mchi, Delta_f_table
                                         , y_array, dlny
                                         , u_array, dlnu):
    integral = dlnu*np.einsum('i,i,ij -> j'
                               , u_array
                               , (3/(1+ms_mchi) * get_beta_n(np.sqrt(ms_mchi)*u_array, n)
                                  - u_array**2 * get_alpha_n(np.sqrt(ms_mchi)*u_array, n))
                               , Delta_f_table) #dim: [y_array]
    dimless_Q_ex = 1/3 /y_array**((n+3)/2) /(1+ms_mchi)**((n-1)/2) * integral
    return dimless_Q_ex

In [ ]:
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])